# Udara Peiris' calibration

## Packages

In [1]:
import requests
import pandas as pd
import os
import numpy as np

## Path and working directory

In [2]:
ROOT = os.path.dirname(os.getcwd())

PATH_DATA = os.path.join(ROOT, 'data')

print(PATH_DATA)

c:\Users\Maude\OneDrive\Documents\etudes\UZH\PhD\2nd year\first semester\calibration in macroeconomics\project\Udara\data


## External estimation of parameters

### GDP growth

5-year GDP growth based on annual GDP per capita in Argentina from the World Bank

In [5]:
# open excel file

df_GDP = pd.read_excel(os.path.join(PATH_DATA, 'GDP_per_capita - Argentina - WB.xlsx'))

#clean database

## delete the last 5 rows because only NaN values
df_GDP = df_GDP.iloc[:-5]

## delete the columns from 1960 to 1982
columns_to_drop = [f"{year} [YR{year}]" for year in range(1960, 1983)]
df_GDP = df_GDP.drop(columns=columns_to_drop)

## convert from wide format to long format
df_long = pd.melt(df_GDP, id_vars=['Country Name', 'Series Name'], var_name='Year', value_name='Value')

## delete the first three columns because useless information as only one country and one variable
df_long = df_long.drop(columns=['Country Name', 'Series Name'])

## delete the first two rows because useless information as only one country and one variable
df_long = df_long.iloc[2:]

## rename the column 'Value' to 'GDP per capita'
df_long = df_long.rename(columns={'Value': 'GDP per capita'})

## change the format of the 'Year' variable
df_long['Year'] = df_long['Year'].str.extract(r'(\d{4})').astype(int)

## create the 'GDP_growth_5' variable
df_long['GDP_growth_5'] = (df_long['GDP per capita'] - df_long['GDP per capita'].shift(5)) / df_long['GDP per capita'].shift(5)

# compute the 25th and 75th percentiles of the 'GDP_growth_5' variable between 1988 and 2001
## Filter the DataFrame for the years between 1988 and 2001
df_filtered = df_long[(df_long['Year'] >= 1988) & (df_long['Year'] <= 2001)]

## Create a vector with the GDP_growth_5 values
gdp_growth_5_vector = df_filtered['GDP_growth_5'].dropna().values

## Calculate the 25th and 75th percentiles
percentile_25 = np.percentile(gdp_growth_5_vector, 25)
percentile_75 = np.percentile(gdp_growth_5_vector, 75)

# Print the percentiles
print(f"the 25th percentile of the 5 year GDP growth between 1988 and 2001: {percentile_25}")
print(f"the 75th percentile of the 5 year GDP growth between 1988 and 2001: {percentile_75}")



the 25th percentile of the 5 year GDP growth between 1988 and 2001: 0.047384516851069064
the 75th percentile of the 5 year GDP growth between 1988 and 2001: 0.6934000884280214
